#order placing sucessful

https://github.com/Komnomnomnom/swigibpy/blob/master/examples/placeorder.py

https://github.com/Komnomnomnom/swigibpy

https://pypi.python.org/pypi/swigibpy/0.4.1

https://pypi.python.org/pypi/swigibpy
    
http://www.swig.org/

In [1]:
import sys
from threading import Event
from swigibpy import (EWrapper, EPosixClientSocket, Contract, Order, TagValue,TagValueList)

In [2]:
MAX_WAIT_SECONDS=10
## This is the reqId IB API sends when a fill is received
FILL_CODE=-1

In [3]:
try:
    # Python 2 compatibility
    input = raw_input
    from Queue import Queue
except:
    from queue import Queue


In [4]:

class PlaceOrderExample(EWrapper):
    '''Callback object passed to TWS, these functions will be called directly by TWS.
    '''

    def __init__(self):
        super(PlaceOrderExample, self).__init__()
        self.order_filled = Event()
        self.order_ids = Queue()

    def openOrderEnd(self):
        '''Not relevant for our example'''
        pass

    def execDetails(self, id, contract, execution):
        '''Not relevant for our example'''
        pass

    def managedAccounts(self, openOrderEnd):
        '''Not relevant for our example'''
        pass

    ###############

    def nextValidId(self, validOrderId):
        '''Capture the next order id'''
        self.order_ids.put(validOrderId)

    def orderStatus(self, id, status, filled, remaining, avgFillPrice, permId,
                    parentId, lastFilledPrice, clientId, whyHeld):

        print(("Order #%s - %s (filled %d, remaining %d, avgFillPrice %f,"
               "last fill price %f)") %
              (id, status, filled, remaining, avgFillPrice, lastFilledPrice))
        if remaining <= 0:
            self.order_filled.set()

    def openOrder(self, orderID, contract, order, orderState):

        print("Order opened for %s" % contract.symbol)

    def commissionReport(self, commissionReport):
        print 'Commission %s %s P&L: %s' % (commissionReport.currency,
                                            commissionReport.commission,
                                            commissionReport.realizedPNL)

In [5]:
prompt = input("WARNING: This example will place an order on your IB "
               "account, are you sure? (Type yes to continue): ")
if prompt.lower() != 'yes':
    sys.exit()

In [6]:
# Instantiate our callback object
callback = PlaceOrderExample()

# Instantiate a socket object, allowing us to call TWS directly. Pass our
# callback object so TWS can respond.
tws = EPosixClientSocket(callback)

In [7]:
# Connect to tws running on localhost
if not tws.eConnect("", 7496,101):
    raise RuntimeError('Failed to connect to TWS')

TWS Warning - 2104: Market data farm connection is OK:usbond
TWS Warning - 2104: Market data farm connection is OK:hfarm
TWS Warning - 2104: Market data farm connection is OK:eufarm
TWS Warning - 2104: Market data farm connection is OK:usfuture
TWS Warning - 2104: Market data farm connection is OK:jfarm
TWS Warning - 2104: Market data farm connection is OK:cashfarm
TWS Warning - 2104: Market data farm connection is OK:usfarm.us
TWS Warning - 2104: Market data farm connection is OK:usfarm
TWS Warning - 2106: HMDS data farm connection is OK:ilhmds
TWS Warning - 2106: HMDS data farm connection is OK:euhmds
TWS Warning - 2106: HMDS data farm connection is OK:fundfarm
TWS Warning - 2106: HMDS data farm connection is OK:ushmds


In [8]:
# Simple contract for GOOG
contract = Contract()
contract.symbol = "SPY"
contract.secType = "STK"
contract.exchange = "SMART"
contract.currency = "USD"

In [9]:
print('Waiting for valid order id')
order_id = callback.order_ids.get(timeout=MAX_WAIT_SECONDS)
if not order_id:
    raise RuntimeError('Failed to receive order id after %ds' % MAX_WAIT_SECONDS)

Waiting for valid order id


In [10]:
# Order details
algoParams = TagValueList()
algoParams.append(TagValue("componentSize", "3"))
algoParams.append(TagValue("timeBetweenOrders", "60"))
algoParams.append(TagValue("randomizeTime20", "1"))
algoParams.append(TagValue("randomizeSize55", "1"))
algoParams.append(TagValue("giveUp", "1"))
algoParams.append(TagValue("catchUp", "1"))
algoParams.append(TagValue("waitForFill", "1"))
algoParams.append(TagValue("startTime", "20110302-14:30:00 GMT"))
algoParams.append(TagValue("endTime", "20110302-21:00:00 GMT"))

In [11]:
order = Order()
order.action = 'BUY'
order.lmtPrice = 214
order.orderType = 'LMT'
order.totalQuantity = 10
order.algoStrategy = "AD"
order.tif = 'DAT'
order.algoParams = algoParams
#order.transmit = False

In [12]:
print("Placing order for %d %s's (id: %d)" % (order.totalQuantity,
                                              contract.symbol, order_id))

# Place the order
tws.placeOrder(
    order_id,                                   # orderId,
    contract,                                   # contract,
    order                                       # order
)

print("\n====================================================================")
print(" Order placed, waiting %ds for TWS responses" % WAIT_TIME)
print("====================================================================\n")


print("Waiting for order to be filled..")

try:
    callback.order_filled.wait(WAIT_TIME)
except KeyboardInterrupt:
    pass
finally:
    if not callback.order_filled.is_set():
        print('Failed to fill order')

    print("\nDisconnecting...")
    tws.eDisconnect()

Placing order for 10 SPY's (id: 1)

 Order placed, waiting 10s for TWS responses

Waiting for order to be filled..
Order opened for SPY
Order #1 - PreSubmitted (filled 0, remaining 10, avgFillPrice 0.000000,last fill price 0.000000)
Failed to fill order

Disconnecting...


TWS Client Error - 503: The TWS is out of date and must be upgraded.
